### Using BERT to solve NLP task: extractive Q&A
* Given two sequences, a question and a context, within that context, BERT extracts our answer to the question.
* https://youtu.be/qgaM0weJHpA

### Examples of extractive QA
1.  "Where do I live?" "I live in Tokyo".  Output: "Tokyo"

### Model1 from Hugging Face: RoBERTa-based-SQuAD2
* https://huggingface.co/deepset/roberta-base-squad2
* Language model: roberta-base
* Training data: SQuAD 2.0
* Evaluating data: SQuAD 2.0
* Infrastructure: 4x Tesla v100

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.8 MB/s eta 0:00:00


In [4]:
from transformers import pipeline, BertForQuestionAnswering

In [5]:
model_name = "deepset/roberta-base-squad2"
qa = pipeline(model=model_name, tokenizer=model_name, revison="v1.0", task="question-answering")

In [6]:
sequence = "Where does Sean study?", "Sean and Mohammed study at GBC."
qa(*sequence)

{'score': 0.46613094210624695, 'start': 27, 'end': 30, 'answer': 'GBC'}

In [7]:
sequence = "Where does everyone live?", "Everyone study at GBC, but they live out of town."
qa(*sequence)

{'score': 0.3723733127117157, 'start': 37, 'end': 48, 'answer': 'out of town'}

In [12]:
long_text = """
Architecturally, the school has a Catholic character.  Atop the Main Building's gold dome is a golden statue of the Virgin Mary.
Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".  Next to the Main Building is the Basilica of the Sacred Heart.  Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.  It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.  At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
"""
sequence = "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes, France", long_text
qa(*sequence)

{'score': 0.9212461709976196,
 'start': 520,
 'end': 546,
 'answer': 'Saint Bernadette Soubirous'}

### Model2 from Hugging Face: BERT large model (uncased) whole word masking finetuned on SQuAD

In [9]:
squad_pipe = pipeline("question-answering", "bert-large-uncased-whole-word-masking-finetuned-squad")

In [10]:
sequence = "Where is the captial of Canada?", "The capital of Canada is Ottawa."
squad_pipe(*sequence)

{'score': 0.6013672947883606, 'start': 25, 'end': 31, 'answer': 'Ottawa'}

In [13]:
# wikipedia https://en.wikipedia.org/wiki/Toronto

long_text = """
During the American Revolutionary War, an influx of British settlers arrived there as United Empire Loyalists fled for the British-controlled
lands north of Lake Ontario. The Crown granted them land to compensate for their losses in the Thirteen Colonies. The new province of
Upper Canada was being created and needed a capital. In 1787, the British Lord Dorchester arranged for the Toronto Purchase with the
Mississaugas of the New Credit First Nation, thereby securing more than a quarter of a million acres (1000 km2) of land in the Toronto area.
Dorchester intended the location to be named Toronto. The first 25 years after the Toronto purchase were quiet, although
"there were occasional independent fur traders" present in the area, with the usual complaints of debauchery and drunkenness.
"""
sequence = "When was the new province of Upper Canada created?", long_text
squad_pipe(*sequence)

{'score': 0.7342036366462708, 'start': 333, 'end': 337, 'answer': '1787'}

### Fine-Tuning a pre-trained model

In [ ]:
# from transformers import DistilBertTokenizerFast, RemBertForQuestionAnswering, pipeline, \
#                        DataCollatorWithPadding, TrainingArguments, Trainer, \
#                        AutoModelForQuestionAnswering, AutoTokenizer

In [ ]:
from transformers import BertTokenizerFast, BertForQuestionAnswering

In [ ]:
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased', return_token_type_ids=True)

qa_bert = BertForQuestionAnswering.from_pretrained('bert-large-uncased')

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-uncased

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
qa_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/data/qa.csv')

qa_df.shape

(29989, 5)

In [ ]:
qa_df.head()

,question,context,start_positions,end_positions,answer
0,What sare the benifts of the blood brain barrir?,Another approach to brain function is to exami...,56,60,isolated from the bloodstream
1,What is surrounded by cerebrospinal fluid?,Another approach to brain function is to exami...,16,16,brain
2,What does the skull protect?,Another approach to brain function is to exami...,11,11,brain
3,What has been injected into rats to produce pr...,Another approach to brain function is to exami...,153,153,chemicals
4,What can cause issues with how the brain works?,Another approach to brain function is to exami...,93,94,brain damage


In [ ]:
qa_df.iloc[0]

question            What sare the benifts of the blood brain barrir?
context            Another approach to brain function is to exami...
start_positions                                                   56
end_positions                                                     60
answer                                 isolated from the bloodstream
Name: 0, dtype: object

In [ ]:
# index 56, 57, 58, 59, and 60 including questin while encoding
bert_tokenizer.decode(bert_tokenizer.encode(qa_df.iloc[0].question, qa_df.iloc[0].context)[56:61])

'isolated from the bloodstream'

In [ ]:
# only grab 4,000 examples
qa_dataset = Dataset.from_pandas(qa_df.sample(4000, random_state=42))

# Dataset has a built in train test split method
qa_dataset = qa_dataset.train_test_split(test_size=0.2)

In [ ]:
# standard preprocessing here with truncation on to truncate longer text
def preprocess(data):
    return bert_tokenizer(data['question'], data['context'], truncation=True)

qa_dataset = qa_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
# freeze all but the last 2 encoder layers in BERT to speed up training
for name, param in qa_bert.bert.named_parameters():
    if 'encoder.layer.22' in name:
        break
    param.requires_grad = False  # disable training in BERT

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer)

In [ ]:
!mkdir qa
!mkdir qa/results
!mkdir qa/logs

In [ ]:
batch_size = 32
epochs = 2

training_args = TrainingArguments(
    output_dir='./qa/results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_dir='./qa/logs',
    save_strategy='epoch',
    logging_steps=10,
    evaluation_strategy='epoch',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=qa_bert,
    args=training_args,
    train_dataset=qa_dataset['train'],
    eval_dataset=qa_dataset['test'],
    data_collator=data_collator
)

# Get initial metrics
trainer.evaluate()

{'eval_loss': 5.842594623565674,
 'eval_runtime': 12.4713,
 'eval_samples_per_second': 64.147,
 'eval_steps_per_second': 2.005}

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,4.424900,4.338307
2,4.039300,4.225910


TrainOutput(global_step=200, training_loss=4.465833511352539, metrics={'train_runtime': 149.8132, 'train_samples_per_second': 42.72, 'train_steps_per_second': 1.335, 'total_flos': 4216561921009152.0, 'train_loss': 4.465833511352539, 'epoch': 2.0})

In [ ]:
# Q/A models are very large and take a long time to fine-tune

In [ ]:
trainer.save_model()

In [ ]:
pipe = pipeline("question-answering", './qa/results', tokenizer=bert_tokenizer)

In [ ]:
pipe("Where is Sinan living these days?", "Sinan lives in California but Matt lives in Boston.")

{'score': 0.1024918332695961, 'start': 15, 'end': 25, 'answer': 'California'}

In [ ]:
princeton = """In 1675, a Quaker missionary from England, encouraged by New Jersey proprietors John Lord 
              "Berkeley and Sir George Carteret, arrived to establish a settlement in this area near the 
              "Delaware River, which was inhabited by the Lenni-Lenape Indians. The Keith survey of 1685 
              "established the western boundary of Middlesex and Somerset Counties and later, the Township 
              "of Princeton. Today Keith's Line is recognized as Province Line Road. With the laying of the 
              "cornerstone for Nassau Hall in 1754, Princeton began its development as a location for 
              "quality education. Nassau Hall was named for William III, Prince of Orange-Nassau. This simple stone 
              "edifice was one of the largest public buildings in the colonies and became a model for many other 
              "structures in New Jersey and Pennsylvania."""

pipe("What survey led to the founding of Princeton?", princeton)

{'score': 0.0021067694760859013,
 'start': 281,
 'end': 385,
 'answer': 'Keith survey of 1685 \n              "established the western boundary of Middlesex and Somerset Counties'}

In [ ]:
# From Huggingface: https://huggingface.co/bert-large-uncased-whole-word-masking-finetuned-squad

squad_pipe = pipeline("question-answering", "bert-large-uncased-whole-word-masking-finetuned-squad")

In [ ]:
squad_pipe("Where is Sinan living these days?", "Sinan lives in California but Matt lives in Boston.")

{'score': 0.9924461245536804, 'start': 15, 'end': 25, 'answer': 'California'}

In [ ]:
squad_pipe("What survey led to the founding of Princeton?", princeton)

{'score': 0.3114534914493561,
 'start': 277,
 'end': 293,
 'answer': 'The Keith survey'}